In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import functions as F
from pyspark.sql.functions import (
    when, countDistinct, count, avg,
    split, explode, trim, lit, length,
    current_date, row_number, regexp_replace,
    to_date, coalesce, col, when,
    year, month, dayofmonth, array_contains, min, max, year
)
from pyspark.sql.functions import sum as spark_sum
from pyspark.sql.window import Window
import numpy as np

In [ ]:
# to read the data set with previous tranformations
df = spark.read.parquet("/FileStore/tables/Imdb_Movie_Dataset-2.csv")
display(df)

In [ ]:
# how many rows do we have
total_rows = df.count()
print(f"Total rows in `df`: {total_rows}")

In [ ]:
# To check the type of each feature
df.printSchema()

%md
## Categorical Feature Analysis

%md
All numerical variables have been analyzed.

%md
### TITLE

In [ ]:
# Safe copy of the original DataFrame
df_copy3 = df.select("*")

In [ ]:
# Count null, empty, and "None"/"none" titles
null_count = df_copy3.filter(col("title").isNull()).count()
empty_count = df_copy3.filter((col("title") == "") | (col("title") == " ")).count()
none_count = df_copy3.filter((col("title") == "None") | (col("title") == "none")).count()

print(f"Null titles: {null_count}")
print(f"Empty titles: {empty_count}")
print(f"None titles: {none_count}")

In [ ]:
# Title length distribution
title_len_df = df_copy3.withColumn("title_length", length(col("title")))
min_len = title_len_df.agg({"title_length": "min"}).collect()[0][0]
max_len = title_len_df.agg({"title_length": "max"}).collect()[0][0]
median_len = title_len_df.approxQuantile("title_length", [0.5], 0.01)[0]
print(f"Title length — min: {min_len}, median: {median_len:.0f}, max: {max_len}")

# Show extremes
print("Longest titles:")
display(
    title_len_df
    .orderBy(col("title_length").desc())
    .select("id", "title", "title_length")
    .limit(5)
)
print("Shortest non-empty titles:")
display(
    title_len_df
    .filter(col("title").isNotNull() & (col("title") != ""))
    .orderBy(col("title_length").asc())
    .select("id", "title", "title_length")
    .limit(5)
)


%md
`title` column does not have any missing values and is stored with the correct data type. However, we noticed that some movies have titles consisting of only a single character. While this is unusual, it is not impossible in the context of cinema. Therefore, we will investigate this further to understand how many such cases exist in the dataset.

In [ ]:
# Count the number of films with just one character as title
single_char_titles_count = df_copy3.filter(length(col("title")) == 1).count()

print(f"Number of movies with single-character titles: {single_char_titles_count}")

# display them
display(df_copy3.filter(length(col("title")) == 1))

%md
Although it may seem unusual for a film to have a single-character title, we have decided to keep these entries in our dataset. Despite their atypical titles, these films contain relevant information for most of our features and therefore remain valuable for our analysis."

%md
### STATUS

%md
Let's check for distinct values 

In [ ]:
# Count distinct non-null statuses
distinct_status_count = df_copy3.filter(col("status").isNotNull()).select("status").distinct().count()
print(f"Number of distinct 'status' values (excluding nulls): {distinct_status_count}")

# Show frequency of each status
status_counts = df_copy3.groupBy("status").count().orderBy(col("count").desc())
print("Counts per status:")
display(status_counts)

In [ ]:
# Calculate average and median revenue by status
status_revenue_stats = (
    df_copy3
    .groupBy("status")
    .agg(
        F.avg("revenue").alias("average_revenue"),
        F.expr("percentile_approx(revenue, 0.5)").alias("median_revenue")
    )
    .orderBy("status")
)

# Display the results
display(status_revenue_stats)

In [ ]:
total_count = df_copy3.count()
non_released_count = df_copy3.filter(col("status") != "Released").count()
percentage = (non_released_count / total_count) * 100

print(f"Non-released movies: {non_released_count} ({percentage:.2f}%)")

In [ ]:
# display them
display(df_copy3.filter(col("status") != "Released"))

%md
These entries represent less than 1% of the dataset and contain incomplete or unreliable information for modeling. Since the goal is to analyze actual outcomes, we will retain only films with Released status.

In [ ]:
# to remove these entries
df_copy3 = df_copy3.filter(col("status") == "Released")

%md
Since at this point all the movies in our dataset are already released,  which was our objective from the beginning,  this column becomes completely unnecessary as it now contains only a single unique value. Therefore, we will remove it from our DataFrame.

In [ ]:
# to remove the column status
df_copy3 = df_copy3.drop("status")

# just to check
df_copy3.printSchema()

%md
### ADULT

In [ ]:
# Check distinct values in the "adult" column and count their occurrences
distinct_adult = df_copy3.groupBy("adult").count()

# Get the number of distinct values in the "adult" column
distinct_adult_count = distinct_adult.count()
print(f"Number of distinct values in 'adult': {distinct_adult_count}")

# Display the distinct values along with their counts
display(distinct_adult)

In [ ]:
# Plot distribution of 'adult' values

# Count occurrences of each distinct value in 'adult' column
adult_distribution = df_copy3.groupBy("adult").count().toPandas()

# Plot the distribution
plt.figure(figsize=(8, 6))
sns.barplot(x='adult', y='count', data=adult_distribution, palette='Set2')
plt.title('Distribution of Adult Movies')
plt.xlabel('Adult')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

%md
Overwhelming majority of the movies is **not** for adults only

%md
### ORIGINAL_LANGUAGE

In [ ]:
# Summary of null, empty-string, "None"/"none", and zero counts for original_language
for cname in ["original_language"]:
    nulls   = df_copy3.filter(col(cname).isNull()).count()
    empties = df_copy3.filter((col(cname) == "") |(col(cname) == " ")).count()
    nones   = df_copy3.filter(col(cname).isin("None", "none")).count()
    zeros   = df_copy3.filter(col(cname) == "0").count()

    print(
        f"{cname}: nulls={nulls}, "
        f"empty strings={empties}, "
        f"'None'/'none' strings={nones}, "
        f"zero values={zeros}"
    )

In [ ]:
# Print number of unique "original_language"
unique_lang_count = df_copy3.select("original_language").distinct().count()
print(f"Number of unique original_language: {unique_lang_count}")

In [ ]:
# Print unique "original_language" values and their counts
df_copy3.groupBy("original_language") \
  .count() \
  .orderBy(col("count").desc()) \
  .show(90, False) # number of rows to display, turn off truncation to see full column values

%md
The listed short forms of languages refer to:
- **tn**: Tswana
- **zh**: Chinese
- **ko**: Korean
- **hi**: Hindi

https://www.science.co.il/language/Codes.php

In [ ]:
# Group by original_language and count occurrences
language_counts = df_copy3.groupBy("original_language").count().orderBy(col("count").desc())

# Show top 5 languages by count
top_languages = language_counts.limit(10).toPandas()

# Plot top 5 languages
plt.figure(figsize=(15, 9))
sns.barplot(x="count", y="original_language", data=top_languages, palette="Set2")
plt.title('Top 10 Most Popular Languages')
plt.xlabel('Number of Movies')
plt.ylabel('Language')
plt.tight_layout()
plt.show()

In [ ]:
# Count the number of English-language movies
count_en = df_copy3.filter(col("original_language") == "en").count()

# Count the number of Spanish-language movies
count_es = df_copy3.filter(col("original_language") == "es").count()

# Calculate the percentage of Spanish movies compared to English
percentage_es_vs_en = (count_es / count_en) * 100

# Print the result
print(f"Spanish-language movies represent {percentage_es_vs_en:.2f}% of English-language movies.")

%md
Great majority of the movies was filmed in english as the original language. To reduce cardinality we will create a new column `en_bool`, where 1 will denote the original language was english and 0 for other original language

In [ ]:
# Create a boolean (0/1) column 'en_bool' for whether original_language == 'en'
df_copy3 = df_copy3.withColumn(
    "en_bool",
    when(col("original_language") == "en", lit(1)).otherwise(lit(0))
)

# Verify
df_copy3.select("original_language", "en_bool").show(10, False)

In [ ]:
# Compute distribution of values in "en_bool"
dist_df = df_copy3.groupBy("en_bool").count().orderBy("en_bool")

# Show the distribution counts
dist_df.show()

%md
We now have a variable that shows good balance and strong relevance for our predictive model.

In [ ]:
# Convert to Pandas for plotting
dist_pd = dist_df.toPandas()

# Plot the distribution
plt.figure(figsize=(6, 4))
plt.bar(dist_pd["en_bool"].astype(str), dist_pd["count"])
plt.xlabel("en_bool (1 = English, 0 = Other)")
plt.ylabel("Count")
plt.title("Distribution of en_bool")
plt.tight_layout()
plt.show()

In [ ]:
# Group by original_language and calculate the average revenue
language_revenue_avg = df_copy3.groupBy("original_language") \
    .agg(F.avg("revenue").alias("average_revenue"))

# Sort the languages by average revenue in descending order
language_revenue_avg = language_revenue_avg.orderBy(col("average_revenue").desc()).limit(5)

# Convert to Pandas for visualization
language_revenue_avg_pd = language_revenue_avg.toPandas()

# 4) Create a pretty bar plot
plt.figure(figsize=(10, 6))
sns.barplot(
    x="average_revenue", 
    y="original_language", 
    data=language_revenue_avg_pd, 
    palette="viridis"  # Color palette
)

# Add titles and labels for clarity
plt.title('Top 5 Original Languages by Average Revenue', fontsize=16)
plt.xlabel('Average Revenue', fontsize=12)
plt.ylabel('Original Language', fontsize=12)
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Group by en_bool and compute average revenue
avg_rev_en = df_copy3.groupBy("en_bool").agg(F.avg("revenue").alias("average_revenue"))

# Convert to Pandas
avg_rev_en_pd = avg_rev_en.toPandas()

# Simple plot
plt.bar(avg_rev_en_pd["en_bool"].astype(str), avg_rev_en_pd["average_revenue"])
plt.xlabel("en_bool (1 = English, 0 = Other)")
plt.ylabel("Average Revenue")
plt.title("Average Revenue by en_bool")
plt.tight_layout()
plt.show()

%md
As expected, the majority of the movies are in English, which also explains the higher overall revenue associated with this language.

%md
### ORIGINAL_TITLE

In [ ]:
# Count the total number of rows
total_rows = df_copy3.count()

# Count the number of rows where the original_title is different from title
different_titles_count = df_copy3.filter(col("original_title") != col("title")).count()

# Calculate the percentage of rows where original_title is different from title
percentage_different_titles = (different_titles_count / total_rows) * 100

print(f"Total rows: {total_rows}")
print(f"Number of movies with different 'original_title' than 'title': {different_titles_count}")
print(f"Percentage of movies with different titles: {percentage_different_titles:.2f}%")

In [ ]:
# Summary of null, empty-string, "None"/"none", and zero counts for 'original_title' column

nulls   = df_copy3.filter(col("original_title").isNull()).count()
empties = df_copy3.filter((col("original_title") == "") | (col("original_title") == " ")).count()
nones   = df_copy3.filter(col("original_title").isin("None", "none")).count()
zeros   = df_copy3.filter(col("original_title") == "0").count()

print(f"original_title: nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Add a column with the length of each original_title
df_length = df_copy3.withColumn("original_title_length", length(col("original_title")))

# Compute min, max, avg, and median of original_title_length
stats = (
    df_length
    .agg(
        F.min("original_title_length").alias("min_length"),
        F.max("original_title_length").alias("max_length"),
        F.avg("original_title_length").alias("avg_length"),
        F.expr("percentile_approx(original_title_length, 0.5)").alias("median_length")
    )
    .collect()[0]
)

min_length    = stats["min_length"]
max_length    = stats["max_length"]
avg_length    = stats["avg_length"]
median_length = stats["median_length"]

print(f"Shortest original_title length: {min_length}")
print(f"Longest original_title length : {max_length}")
print(f"Average original_title length : {avg_length:.1f}")
print(f"Median original_title length  : {median_length}")

# Show examples of the shortest and longest original_titles
print("Examples of shortest original_titles:")
display(
    df_length
      .filter(col("original_title_length") == min_length)
      .select("id", "title", "original_title", "original_title_length")
      .limit(5)
)

print("Examples of longest original_titles:")
display(
    df_length
      .filter(col("original_title_length") == max_length)
      .select("id", "title", "original_title", "original_title_length")
      .limit(5)
)


%md
This category has too much diversity and will not add meaningful value to our model or to the main objective of this project. We will remove it.

In [ ]:
df_copy3 = df_copy3.drop("original_title")

%md
### OVERVIEW

In [ ]:
# Summary of null, empty-string, "None"/"none", and zero counts for 'overview' column
total = df_copy3.count()

nulls   = df_copy3.filter(col("overview").isNull()).count()
empties = df_copy3.filter((col("overview") == "") | (col("overview") == " ")).count()
nones   = df_copy3.filter(col("overview").isin("None", "none")).count()
zeros   = df_copy3.filter(col("overview") == "0").count()

print(f"overview: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Set those empty or whitespace-only overviews to null
df_copy3 = df_copy3.withColumn(
    "overview",
    when(trim(col("overview")) == "", None).otherwise(col("overview"))
)

# Then check again how many empty strings there are
empties = df_copy3.filter((col("overview") == "") |(col("overview") == " ")).count()
print(f"Number of empty strings left: {empties}")

In [ ]:
# Add a column with the length of each overview
df_length = df_copy3.withColumn("overview_length", length(col("overview")))

# Compute min, max, avg, and median of overview_length
stats = (
    df_length
    .agg(
        F.min("overview_length").alias("min_length"),
        F.max("overview_length").alias("max_length"),
        F.avg("overview_length").alias("avg_length"),
        F.expr("percentile_approx(overview_length, 0.5)").alias("median_length")
    )
    .collect()[0]
)

min_length    = stats["min_length"]
max_length    = stats["max_length"]
avg_length    = stats["avg_length"]
median_length = stats["median_length"]

print(f"Shortest overview length: {min_length}")
print(f"Longest overview length : {max_length}")
print(f"Average overview length : {avg_length:.1f}")
print(f"Median overview length  : {median_length}")

# Show examples of the shortest and longest overviews
print("Examples of shortest overviews:")
display(
    df_length
      .filter(col("overview_length") == min_length)
      .select("id", "title", "overview", "overview_length")
      .limit(5)
)

print("Examples of longest overviews:")
display(
    df_length
      .filter(col("overview_length") == max_length)
      .select("id", "title", "overview", "overview_length")
      .limit(5)
)


In [ ]:
# Plot the distribution of overview lengths
overview_lengths_pd = df_length.select("overview_length").toPandas()

plt.figure(figsize=(10, 6))
sns.histplot(overview_lengths_pd["overview_length"], bins=50, kde=False)
plt.title("Distribution of Overview Lengths")
plt.xlabel("Overview Length (characters)")
plt.ylabel("Number of Movies")
plt.tight_layout()
plt.show()

%md
Distribution of overview leghts is right skewed. It indicates that most overviews have around 150 words. Only some have the maximum number of 998 characters.

In [ ]:
df_copy3 = df_copy3.drop("overview_length")

%md
### TAGLINE

%md
The column `tagline` has the correct data type - string. Let's investigate this feature:

In [ ]:
# Summary for 'tagline'
total = df_copy3.count()

nulls   = df_copy3.filter(col("tagline").isNull()).count()
empties = df_copy3.filter((col("tagline") == "") | (col("tagline") == " ")).count()
nones   = df_copy3.filter(col("tagline").isin("None", "none")).count()
zeros   = df_copy3.filter(col("tagline") == "0").count()

print(f"tagline: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Replace "None"/"none" in `tagline` with null

# Count before replacement
none_count = df.filter(col("tagline").isin("None", "none")).count()
print(f"Taglines with 'None' or 'none' before cleanup: {none_count}")

# Perform replacement
df = df.withColumn(
    "tagline",
    when(col("tagline").isin("None", "none"), None).otherwise(col("tagline"))
)

# Verify post-replacement
post_none_count = df.filter(col("tagline").isNull()).count()
print(f"Taglines null after cleanup: {post_none_count}")

In [ ]:
# Compute min, max, average, and median lengths of `tagline`
from pyspark.sql.functions import length, avg, expr, min as spark_min, max as spark_max

df_length = df_copy3.withColumn("tagline_length", length(col("tagline")))

stats = df_length.agg(
    spark_min("tagline_length").alias("min_length"),
    spark_max("tagline_length").alias("max_length"),
    avg("tagline_length").alias("avg_length"),
    expr("percentile_approx(tagline_length, 0.5)").alias("median_length")
).collect()[0]

print(f"Shortest tagline: {stats['min_length']}")
print(f"Longest  tagline: {stats['max_length']}")
print(f"Average  tagline: {stats['avg_length']:.1f}")
print(f"Median   tagline: {stats['median_length']}")

In [ ]:
# Plot distribution of tagline lengths
# Convert to Pandas
lengths_pd = df_length.select("tagline_length").toPandas()["tagline_length"].dropna()

plt.figure(figsize=(10, 6))
sns.histplot(lengths_pd, bins=50, kde=False)
plt.title("Distribution of Tagline Lengths")
plt.xlabel("Tagline Length (chars)")
plt.ylabel("Number of Movies")
plt.tight_layout()
plt.show()

In [ ]:
# Compare average revenue: tagline NULL vs non-NULL
# Compute stats
rev_stats = (
    df.groupBy(col("tagline").isNull().alias("tagline_null"))
      .agg(
          F.avg("revenue").alias("avg_revenue"),
          F.expr("percentile_approx(revenue, 0.5)").alias("median_revenue")
      )
      .orderBy("tagline_null")
      .toPandas()
)

# Map boolean to labels
rev_stats["tagline_null"] = rev_stats["tagline_null"].map({True: "Null Tagline", False: "Has Tagline"})

plt.figure(figsize=(8, 6))
sns.barplot(x="tagline_null", y="avg_revenue", data=rev_stats, color="skyblue", label="Average Revenue", ci=None)
sns.barplot(x="tagline_null", y="median_revenue", data=rev_stats, color="orange", label="Median Revenue", ci=None)
plt.title("Revenue by Tagline Presence")
plt.xlabel("")
plt.ylabel("Revenue")
plt.legend()
plt.tight_layout()
plt.show()

%md
To reduce complexity of this variable we will create a boolean feature "tagline_bool", for which value 1 will denote that the tagline was present and 0 for null taglines

In [ ]:
# Create boolean column from "tagline" (0 if null, 1 if not)
df_copy3 = df_copy3.withColumn(
    "tagline_bool",
    when(col("tagline").isNull(), lit(0)).otherwise(lit(1))
)

In [ ]:
# Show counts of tagline_bool values
df_copy3.groupBy("tagline_bool") \
  .count() \
  .orderBy("tagline_bool") \
  .show()

#  Convert to Pandas for plotting
dist_pd = (
    df_copy3.groupBy("tagline_bool")
      .count()
      .orderBy("tagline_bool")
      .toPandas()
)

# Plot the distribution
plt.figure(figsize=(6, 4))
plt.bar(dist_pd["tagline_bool"].astype(str), dist_pd["count"])
plt.xlabel("tagline_bool (1 = has tagline, 0 = no tagline)")
plt.ylabel("Count")
plt.title("Distribution of Tagline Presence")
plt.tight_layout()
plt.show()

%md
Now that the `tagline` feature has been converted to a boolean, it becomes useful for our predictive model

%md
### GENRES

In [ ]:
# Summary for 'genres'
total = df_copy3.count()

nulls   = df_copy3.filter(col("genres").isNull()).count()
empties = df_copy3.filter((col("genres") == "") | (col("genres") == " ")).count()
nones   = df_copy3.filter(col("genres").isin("None", "none")).count()
zeros   = df_copy3.filter(col("genres") == "0").count()

print(f"genres: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
df_copy3.select("genres").show(50, truncate=False)

In [ ]:
# Count occurrences of each distinct genre
# Split the comma-separated string into an array, trimming whitespace
df_copy3 = df_copy3.withColumn(
    "genres_array",
    split(col("genres"), "\\s*,\\s*")
)

# Explode the array so each genre gets its own row
genres_exploded = df_copy3.select(
    explode(col("genres_array")).alias("genre")
)

# Group by the individual genre values and count
genres_exploded.groupBy("genre") \
    .count() \
    .orderBy(col("count").desc()) \
    .show(100, False)  # adjust the number 100 if you want more or fewer rows

# Count the number of distinct genres
distinct_genre_count = genres_exploded.select("genre").distinct().count()
print(f"Number of distinct genres: {distinct_genre_count}")

In [ ]:
# Handle "genres" for boolean columns
# Explode into individual genres and plot top 10 by total revenue
df_copy3 = df_copy3.withColumn(
    "genres_array",
    split(regexp_replace(col("genres"), "\\s*,\\s*", ","), ",")
)

genre_rev_df = (
    df_copy3.select(explode(col("genres_array")).alias("genre"), col("revenue"))
      .groupBy("genre")
      .agg(spark_sum("revenue").alias("total_revenue"))
      .orderBy(col("total_revenue").desc())
)

genre_rev_df.show(10)

In [ ]:
top10_genres = genre_rev_df.limit(10).toPandas()
plt.figure(figsize=(10,6))
plt.bar(top10_genres["genre"], top10_genres["total_revenue"])
plt.xticks(rotation=45, ha='right')
plt.title("Top 10 Genres by Total Revenue")
plt.xlabel("Genre")
plt.ylabel("Total Revenue")
plt.tight_layout()
plt.show()

%md
Here we can clearly observe that the genres contributing to higher movie revenue are Action, Adventure, Comedy, and Drama, standing out significantly from the rest.

%md
To simplify this variable and make it more usable, we will transform it into six possible boolean combinations based on the five most common genres. If none of these genres are present (i.e., all five are set to 0), the movie will be classified as belonging to one of the remaining, less frequent genres.

In [ ]:
# Create boolean columns for top 5 most profitable genres
top5_genres = [row["genre"] for row in genre_rev_df.limit(5).collect()]
for g in top5_genres:
    col_name = g.lower().replace(" ", "_")
    df_copy3 = df_copy3.withColumn(
        col_name,
        when(array_contains(col("genres_array"), g), lit(1)).otherwise(lit(0))
    )

In [ ]:
# recover the list of boolean columns
bool_cols   = [g.lower().replace(" ", "_") for g in top5_genres]

# View the first 10 rows of those boolean columns
print("Boolean columns for top 5 genres:", bool_cols)
df_copy3.select(bool_cols).show(10, False)

# Count how many movies have each genre (sum of the 1s)
counts_df = df_copy3.select([spark_sum(col(c)).alias(c) for c in bool_cols]) \
              .toPandas() \
              .melt(var_name="genre", value_name="count")

# Plot the distribution
plt.figure(figsize=(8, 5))
plt.bar(counts_df["genre"], counts_df["count"])
plt.xlabel("Genre")
plt.ylabel("Number of Movies")
plt.title("Distribution of Top 5 Genres")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

%md
From this chart, and based on what we already knew, we can see that the Adventure genre, although it is the least represented among the five most popular genres in our dataset, ends up having the second highest average revenue. This highlights its strong impact on box office performance.

%md
### PRODUCTION_COMPANIES

In [ ]:
# Summary for 'production_companies'
total = df_copy3.count()

nulls   = df_copy3.filter(col("production_companies").isNull()).count()
empties = df_copy3.filter((col("production_companies") == "") | (col("production_companies") == " ")).count()
nones   = df_copy3.filter(col("production_companies").isin("None", "none")).count()
zeros   = df_copy3.filter(col("production_companies") == "0").count()

print(f"production_companies: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Top 10 production_companies by number of movies
company_counts = (
    df_copy3
    .groupBy("production_companies")
    .count()
    .orderBy(col("count").desc())
    .limit(10)
    .toPandas()
)

plt.figure(figsize=(10,6))
sns.barplot(x="count", y="production_companies", data=company_counts, palette="Greens_d")
plt.title("Top 10 Production Companies by Number of Movies")
plt.xlabel("Number of Movies")
plt.ylabel("Company")
plt.tight_layout()
plt.show()


In [ ]:
# Top 10 production_companies by average revenue

company_revenue = (
    df_copy3
    .groupBy("production_companies")
    .agg(avg("revenue").alias("avg_revenue"))
    .orderBy(col("avg_revenue").desc())
    .limit(10)
    .toPandas()
)

plt.figure(figsize=(10,6))
sns.barplot(x="avg_revenue", y="production_companies", data=company_revenue, palette="Oranges_d")
plt.title("Top 10 Production Companies by Average Revenue")
plt.xlabel("Average Revenue")
plt.ylabel("Company")
plt.tight_layout()
plt.show()


%md
From these two charts, it is easy to see that the number of films produced does not necessarily reflect quality or success. For example, Metro-Goldwyn-Mayer is the company with the highest number of productions, yet when looking at the top 10 production companies by revenue, it ranks only 10th. This highlights the discrepancy between quantity and profitability.

%md
There are many high-cardinality cells in `production_companies`. Therefore, we will create boolean columns, for the top 6 most profitable ones

In [ ]:
# Split & explode high‐cardinality “production_companies”
df_copy3 = df_copy3.withColumn(
    "prod_companies_array",
    split(
        regexp_replace(col("production_companies"), r"\s*,\s*", ","),  # normalize commas
        ","
    )
)
prod_exploded = df_copy3.select(
    explode(col("prod_companies_array")).alias("company"),
    col("revenue")
)

In [ ]:
# Count occurrences & distinct count
prod_counts = (
    prod_exploded
      .groupBy("company")
      .count()
      .orderBy(col("count").desc())
)
prod_counts.show(100, False)  # show top 100 by frequency
print(f"Number of distinct production companies: {prod_counts.count()}")

In [ ]:
# Top 10 companies by total revenue & plot
prod_rev = (
    prod_exploded
      .groupBy("company")
      .agg(spark_sum("revenue").alias("total_revenue"))
      .orderBy(col("total_revenue").desc())
)
top10_prod = prod_rev.limit(10)
top10_prod.show(10, False)

# Plot
pd_top10_prod = top10_prod.toPandas()
plt.figure(figsize=(10,6))
plt.bar(pd_top10_prod["company"], pd_top10_prod["total_revenue"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Production Companies by Total Revenue")
plt.xlabel("Production Company")
plt.ylabel("Total Revenue")
plt.tight_layout()
plt.show()

In [ ]:
# Create boolean flags for the top 6 most profitable companies
top6_prod = [row["company"] for row in prod_rev.limit(6).collect()]
for comp in top6_prod:
    flag_col = comp.lower().replace(" ", "_").replace(".", "").replace(",", "")
    df_copy3 = df_copy3.withColumn(
        flag_col,
        when(array_contains(col("prod_companies_array"), comp), 1).otherwise(0)
    )

In [ ]:
# Plot distribution of these boolean flags
bool_cols_prod = [c.lower().replace(" ", "_").replace(".", "").replace(",", "") for c in top6_prod]
dist_prod = (
    df_copy3
      .select([spark_sum(col(c)).alias(c) for c in bool_cols_prod])
      .toPandas()
      .melt(var_name="company", value_name="count")
)
plt.figure(figsize=(8,5))
plt.bar(dist_prod["company"], dist_prod["count"])
plt.xticks(rotation=45, ha="right")
plt.title("Distribution of Top 6 Production Company Flags")
plt.ylabel("Number of Movies")
plt.xlabel("Company Flag")
plt.tight_layout()
plt.show()

%md
For this variable to become useful for our model, it also needs to be converted into a binary system of 6 categories, representing the top 5 production companies by revenue, plus one additional category for all others.

%md
### PRODUCTION_COUNTRIES

In [ ]:
# Summary for 'production_countries'
total = df_copy3.count()

nulls   = df_copy3.filter(col("production_countries").isNull()).count()
empties = df_copy3.filter((col("production_countries") == "") | (col("production_countries") == " ")).count()
nones   = df_copy3.filter(col("production_countries").isin("None", "none")).count()
zeros   = df_copy3.filter(col("production_countries") == "0").count()

print(f"production_countries: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Top 5 production_countries by number of movies

country_counts = (
    df_copy3
    .groupBy("production_countries")
    .count()
    .orderBy(col("count").desc())
    .limit(5)
    .toPandas()
)

plt.figure(figsize=(8,5))
sns.barplot(x="count", y="production_countries", data=country_counts, palette="Blues_d")
plt.title("Top 5 Production Countries by Number of Movies")
plt.xlabel("Number of Movies")
plt.ylabel("Country")
plt.tight_layout()
plt.show()

%md
It is easy to observe that the United States is the largest producer of films. To make this variable useful for our model, we will also convert it into a binary system of five categories, representing the top film-producing countries

%md
### SPOKEN_LANGUAGES

In [ ]:
# Summary for 'spoken_languages'
total = df_copy3.count()

nulls   = df_copy3.filter(col("spoken_languages").isNull()).count()
empties = df_copy3.filter((col("spoken_languages") == "") | (col("spoken_languages") == " ")).count()
nones   = df_copy3.filter(col("spoken_languages").isin("None", "none")).count()
zeros   = df_copy3.filter(col("spoken_languages") == "0").count()

print(f"spoken_languages: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Display distinct languages
display(
    df_copy3
      .select("spoken_languages")
      .distinct()
      .orderBy("spoken_languages")
)

In [ ]:
# Top 5 spoken_languages by number of movies
lang_counts = (
    df_copy3
    .groupBy("spoken_languages")
    .count()
    .orderBy(col("count").desc())
    .limit(5)
    .toPandas()
)

plt.figure(figsize=(8,5))
sns.barplot(x="count", y="spoken_languages", data=lang_counts, palette="Purples_d")
plt.title("Top 5 Spoken Languages by Number of Movies")
plt.xlabel("Number of Movies")
plt.ylabel("Language")
plt.tight_layout()
plt.show()

%md
For this variable as well, we will create a binary system with five elements, representing the four most frequent values along with one additional category for all remaining cases

%md
### KEYWORDS

In [ ]:
# Summary for 'keywords'
total = df_copy3.count()

nulls   = df_copy3.filter(col("keywords").isNull()).count()
empties = df_copy3.filter((col("keywords") == "") | (col("keywords") == " ")).count()
nones   = df_copy3.filter(col("keywords").isin("None", "none")).count()
zeros   = df_copy3.filter(col("keywords") == "0").count()

print(f"keywords: total={total}, nulls={nulls}, empty strings={empties}, 'None'/'none' strings={nones}, zero values={zeros}")

In [ ]:
# Split & explode “keywords”
df_copy3 = df_copy3.withColumn(
    "keywords_array",
    split(
        regexp_replace(col("keywords"), r"\s*,\s*", ","), 
        ","
    )
)
kw_exploded = df_copy3.select(
    explode(col("keywords_array")).alias("keyword"),
    col("revenue")
)

In [ ]:
# Count occurrences & distinct count
kw_counts = (
    kw_exploded
      .groupBy("keyword")
      .count()
      .orderBy(col("count").desc())
)
kw_counts.show(100, False)
print(f"Number of distinct keywords: {kw_counts.count()}")


In [ ]:
# Top 10 keywords by total revenue & plot
kw_rev = (
    kw_exploded
      .groupBy("keyword")
      .agg(spark_sum("revenue").alias("total_revenue"))
      .orderBy(col("total_revenue").desc())
)
top10_kw = kw_rev.limit(10)
top10_kw.show(10, False)

pd_top10_kw = top10_kw.toPandas()
plt.figure(figsize=(10,6))
plt.bar(pd_top10_kw["keyword"], pd_top10_kw["total_revenue"])
plt.xticks(rotation=45, ha="right")
plt.title("Top 10 Keywords by Total Revenue")
plt.xlabel("Keyword")
plt.ylabel("Total Revenue")
plt.tight_layout()
plt.show()

In [ ]:
# Boolean flags for the top 3 most profitable keywords
top3_kw = [row["keyword"] for row in kw_rev.limit(3).collect()]
for kw in top3_kw:
    flag_col = kw.lower().replace(" ", "_").replace(".", "").replace(",", "")
    df_copy3 = df_copy3.withColumn(
        flag_col,
        when(array_contains(col("keywords_array"), kw), 1).otherwise(0)
    )

In [ ]:
# Plot distribution of these boolean keyword flags
bool_cols_kw = [k.lower().replace(" ", "_").replace(".", "").replace(",", "") for k in top3_kw]
dist_kw = (
    df_copy3
      .select([spark_sum(col(c)).alias(c) for c in bool_cols_kw])
      .toPandas()
      .melt(var_name="keyword", value_name="count")
)
plt.figure(figsize=(8,5))
plt.bar(dist_kw["keyword"], dist_kw["count"])
plt.xticks(rotation=45, ha="right")
plt.title("Distribution of Top 3 Keyword Flags")
plt.ylabel("Number of Movies")
plt.xlabel("Keyword Flag")
plt.tight_layout()
plt.show()

In [ ]:
# Explode the 'keywords' string column into individual rows.
#    Adjust the delimiter in split() if your keywords are comma-separated, e.g. "," instead of "\\|"
keywords_exploded = df.select(
    explode(
        split(col("keywords"), "\\|")
    ).alias("keyword")
).filter(col("keyword") != "")

# 2) Count and take the top 50
top50 = (
    keywords_exploded
      .groupBy("keyword")
      .count()
      .orderBy(col("count").desc())
      .limit(50)
      .toPandas()
)

# 3) Plot as a horizontal bar chart with matplotlib
plt.figure(figsize=(10, 12))
plt.barh(top50["keyword"][::-1], top50["count"][::-1])
plt.xlabel("Frequency")
plt.ylabel("Keyword")
plt.title("Top 50 Keywords by Frequency")
plt.tight_layout()
plt.show()


In [ ]:
# 1) Explode keywords and carry revenue along
keywords_df = df.select(
    explode(split(col("keywords"), "\\|")).alias("keyword"),
    col("revenue").cast("double")
).filter(col("keyword") != "")

# 2) Compute frequency and average revenue per keyword
kw_stats = (
    keywords_df
      .groupBy("keyword")
      .agg(
          count("*").alias("frequency"),
          avg("revenue").alias("avg_revenue")
      )
)

# 3) Take top 10 keywords by frequency
top10 = (
    kw_stats
      .orderBy(col("frequency").desc())
      .limit(10)
)

# 4) Show the results in Spark
top10.show(truncate=False)

# 5) (Optional) Convert to Pandas and plot average revenue
pd_top10 = top10.toPandas()

plt.figure(figsize=(10, 6))
plt.bar(pd_top10["keyword"], pd_top10["avg_revenue"])
plt.xlabel("Keyword")
plt.ylabel("Average Revenue")
plt.title("Average Revenue for Top 10 Keywords by Frequency")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


%md
## New Features

%md
### Profit and ROI

In [ ]:
# Filter out rows with zero or null budget or revenue to avoid division errors
df_money = df.filter((col("budget") > 0) & (col("revenue") > 0))

# Create 'profit' and 'roi' columns
df_money = df_money.withColumn("profit", col("revenue") - col("budget"))
df_money = df_money.withColumn("roi", col("revenue") / col("budget"))

In [ ]:
# Describe the statistics for 'profit' and 'roi' columns in PySpark
df_money.describe(['profit', 'roi']).show()

%md
#### Check only de ROI > 0

In [ ]:
# Step 1: Calculate profit and ROI
df_money = df.withColumn("profit", col("revenue") - col("budget"))
df_money = df_money.withColumn("roi", col("revenue") / col("budget"))

# Step 2: Filter for positive profit and ROI
profit_roi_filtered = df_money.filter((col('profit') > 0) & (col('roi') > 0))

# Step 3: Convert to pandas DataFrame for plotting
profit_roi_filtered_pd = profit_roi_filtered.select('profit').toPandas()

# Step 4: Plot the distribution of profit
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 5))
sns.histplot(profit_roi_filtered_pd['profit'], bins=100, color='mediumseagreen')
plt.title('Distribution of Movie Profit')
plt.xlabel('Profit (Revenue - Budget)')
plt.ylabel('Number of Movies')
plt.axvline(0, color='red', linestyle='--', label='Break-even')
plt.legend()
plt.tight_layout()
plt.show()

%md
#### Movies with the highest profit

In [ ]:
# Step 1: Sort by profit in descending order and get the top 10 movies
top_profit = profit_roi_filtered.orderBy(col('profit'), ascending=False).limit(10)

# Step 2: Select the relevant columns (title, revenue, budget, profit)
top_profit_selected = top_profit.select('title', 'revenue', 'budget', 'profit')

# Step 3: Convert to pandas DataFrame to display in a table
top_profit_selected_pd = top_profit_selected.toPandas()

# Step 4: Display the result
print(top_profit_selected_pd)

%md
#### Movies with the highest ROI

In [ ]:
# Step 1: Sort by ROI in descending order and get the top 10 movies
top_roi = profit_roi_filtered.orderBy(col('roi'), ascending=False).limit(10)

# Step 2: Select the relevant columns (title, budget, revenue, roi)
top_roi_selected = top_roi.select('title', 'budget', 'revenue', 'roi')

# Step 3: Convert to pandas DataFrame to display in a table
top_roi_selected_pd = top_roi_selected.toPandas()

# Step 4: Display the result
print(top_roi_selected_pd)


%md
## Correlation Matrix for Popularity, Vote_Count and Revenue

In [ ]:
# Convert to pandas for correlation analysis
df_pd = df.select('popularity', 'vote_count', 'revenue').toPandas()

# Compute correlation matrix
correlation_popularity = df_pd.corr()

# Display the correlation matrix
print("Correlation between Popularity, Vote Count, and Revenue:")
print(correlation_popularity)

%md
Vote count appears to have the strongest relationship with revenue.

Both popularity and vote count are positively correlated with revenue, but the relationship with popularity is weaker compared to the relationship between vote count and revenue.

In [ ]:
# To check the type of each feature
df.printSchema()
# Function to calculate missing values by column
def missing_values_table_spark(df):
    """
    Calculates the total number and percentage of missing (null) values 
    for each column in a PySpark DataFrame.

    Returns a Pandas DataFrame with columns:
    - 'Column': column name
    - 'Missing Values': count of missing values
    - '% of Total Values': percentage of missing values
    
    Only columns with missing values are included, sorted in descending order.
    """
    # Calculate the total missing values for each column
    mis_val = df.select([F.sum(F.col(c).isNull().cast("int")).alias(c) for c in df.columns])

    # Convert to Pandas for easier handling
    mis_val_pd = mis_val.toPandas().transpose()

    # Calculate the percentage of missing values for each column
    mis_val_percent = (mis_val_pd[0] / df.count()) * 100

    # Create a new table combining count and percentage
    mis_val_table = pd.concat([mis_val_pd, mis_val_percent], axis=1)
    mis_val_table.columns = ['Missing Values', '% of Total Values']

    # Keep only columns with >0% missing, sort descending, round
    mis_val_table = (
        mis_val_table[mis_val_table['% of Total Values'] > 0]
        .sort_values('% of Total Values', ascending=False)
        .round(1)
    )

    # Reset index so that original column names become a column
    mis_val_table = mis_val_table.reset_index().rename(columns={'index': 'Column'})

    # Print summary
    print(f"Your selected dataframe has {len(df.columns)} columns.\n"
          f"There are {mis_val_table.shape[0]} columns that have missing values.")

    return mis_val_table

# Usage
missing_values = missing_values_table_spark(df)
display(missing_values)

%md
### Save the DataSet

In [ ]:
df = df_copy3

In [ ]:
# to save our dataset
df.write.mode("overwrite").parquet("/FileStore/tables/Imdb_Movie_Dataset-4.csv")
display(df)